# The "Long Pole" Effect in Gradle's Test Execution

When we assign a number of tests (with different runtimes) to multiple processors for parallel execution, the overall test runtime is that of the processor with the highest load as measured by the summed test execution times. Ideally, this approaches the quotient between total test runtime and number of processors. However, when one processor is assigned an unproportionally high load of tests, the overall runtime will depend on this processor: It will have to continue to work off tasks after all other processors have finshed, thus becoming the "long pole" in the runtime.

This phenomenon has been [discussed for Gradle's parallel test execution](https://github.com/gradle/gradle/issues/2669).

In [39]:
def roundRobin(tests, procCount):
    """ 
    Takes a list of test items and distributes them in a straight-forward round-robin
    fashion amongst the number of given processors.
    :return: a list of `procCount` lists L1 ...Lx, where each Li denotes the tests assigned
             to processor #x.
    """
    procs = [[] for _i in range(procCount)]

    for tIdx, test in enumerate(tests):
        pos = tIdx % procCount
        procs[pos].append(test)
    return procs

In [40]:
def prettyPr(procTestLists):
    """
        Pretty-prints a list of lists of test assignments
    """
    pWidth = 1+len(str(len(procTestLists)))
    tWidth = 1+len(str(max(e for l in procTestLists for e in l)))
    for pIdx, tList in enumerate(procTestLists):        
        print("proc {:>{pWidth}} > {}".format(pIdx, " ".join("{:>{tWidth}}".format(t, tWidth=tWidth) for t in tList),
              pWidth=pWidth))

## Scheduling

Gradle currently uses a round-robin approach to evenly pre-distribute tests to multiple processors. If we have 20 tests and 5 processors, they would be allocated as follows:

In [41]:
prettyPr(roundRobin(range(20), 5))

proc  0 >   0   5  10  15
proc  1 >   1   6  11  16
proc  2 >   2   7  12  17
proc  3 >   3   8  13  18
proc  4 >   4   9  14  19


This allocation is based solely on the order of the tests; it does not take into account size information. E.g., if tests 0, 5, and 10 happen to be very long, processor 0 would become the long pole in this scenario.

In [4]:
def shiftingRoundRobin(tests, procCount):
    """ 
    Takes a list of test items and distributes them in a shifting round-robin
    fashion amongst the number of given processors.
    :return: a list of `procCount` lists L1 ...Lx, where each Li denotes the tests assigned
             to processor #x.
    """
    
    procs = [[] for _i in range(procCount)]

    for tIdx, test in enumerate(tests):
        shift = (tIdx//procCount)%procCount
        pos = (shift + tIdx) % procCount
        procs[pos].append(test)
    return procs

In [37]:
res = shiftingRoundRobin(range(15), 3)
prettyPr(res)

proc  0 >   0   5   7   9  14
proc  1 >   1   3   8  10  12
proc  2 >   2   4   6  11  13
